In [ ]:
!pip install einops

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from torch.utils.checkpoint import checkpoint
import math

class Config:
    def __init__(self):
        self.vocab_size = 32000
        self.d_model = 5120
        self.n_layers = 2
        self.n_heads = 8
        self.d_kv_comp = 128
        self.d_rope = 16
        self.n_experts = 32
        self.n_shared = 2
        self.top_k = 2
        self.seq_len = 256
        self.batch_size = 1
        self.ffn_dim = 384
        self.device_groups = 4 # For device-limited routing

config = Config()

class Expert(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(config.d_model, config.ffn_dim)
        self.w2 = nn.Linear(config.ffn_dim, config.d_model)

    def forward(self, x):
        return self.w2(F.gelu(self.w1(x)))

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, scale=40):
        super().__init__()
        assert dim % 2 == 0, "Dimension must be even for rotary embeddings"
        self.dim = dim
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim//2, 2).float() / (dim//2)))
        self.register_buffer("inv_freq", inv_freq)
        self.scale = 40

    def forward(self, seq_len):
        t = torch.arange(seq_len, device=self.inv_freq.device).type_as(self.inv_freq) / self.scale
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        return torch.cat((freqs, freqs), dim=-1)

def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary(x, cos, sin):
    """
    Apply rotary embeddings to the first half of x.
    """
    # Split x into two parts: one for rotary embeddings and the other untouched
    x_rot, x_base = x.split(cos.shape[-1], dim=-1)
    # Apply rotary embeddings to the rotary part
    x_rot = (x_rot * cos) + (rotate_half(x_rot) * sin)
    # Concatenate the rotary-applied and base parts
    return torch.cat([x_rot, x_base], dim=-1)


class MemoryOptimizedMLA(nn.Module):
    def __init__(self):
        super().__init__()
        self.d_head = config.d_model // config.n_heads
        self.split_dim = self.d_head - config.d_rope

        # Projections
        self.W_dkv = nn.Linear(config.d_model, config.d_kv_comp)
        self.W_dq = nn.Linear(config.d_model, config.d_kv_comp)

        # Changed value projection to use d_head instead of split_dim
        self.W_uk = nn.Linear(config.d_kv_comp, config.n_heads * self.split_dim)
        self.W_uv = nn.Linear(config.d_kv_comp, config.n_heads * self.d_head)
        self.W_uq = nn.Linear(config.d_kv_comp, config.n_heads * self.split_dim)

        self.W_qr = nn.Linear(config.d_kv_comp, config.n_heads * config.d_rope)
        self.W_kr = nn.Linear(config.d_model, config.n_heads * config.d_rope)

        self.rotary = RotaryEmbedding(config.d_rope)
        self.output = nn.Linear(config.n_heads * self.d_head, config.d_model)

    def forward(self, h, past_kv=None):
        batch_size, seq_len, _ = h.shape

        # KV Compression
        c_kv = self.W_dkv(h)
        k = self.W_uk(c_kv).view(batch_size, seq_len, config.n_heads, self.split_dim)
        v = self.W_uv(c_kv).view(batch_size, seq_len, config.n_heads, self.d_head)

        # Query Compression
        c_q = self.W_dq(h)
        q_base = self.W_uq(c_q).view(batch_size, seq_len, config.n_heads, self.split_dim)
        q_rot = self.W_qr(c_q).view(batch_size, seq_len, config.n_heads, config.d_rope)

        # Rotary embeddings with proper dimensions
        rotary_emb = self.rotary(seq_len)
        cos = torch.cos(rotary_emb).view(1, seq_len, 1, -1)  # [1, seq, 1, dim]
        sin = torch.sin(rotary_emb).view(1, seq_len, 1, -1)

        # Apply rotary embeddings
        q_rot = apply_rotary(q_rot, cos, sin)
        k_rot = apply_rotary(
            self.W_kr(h).view(batch_size, seq_len, config.n_heads, config.d_rope),
            cos, sin
        )

        q = torch.cat([q_base, q_rot], dim=-1)
        k = torch.cat([k, k_rot], dim=-1)

        # Attention computation
        scores = torch.einsum("bqhd,bkhd->bhqk", q, k) / math.sqrt(self.d_head)
        attn = F.softmax(scores, dim=-1)
        out = torch.einsum("bhqk,bkhd->bqhd", attn, v)

        return self.output(out.contiguous().view(batch_size, seq_len, -1)), (c_kv, k_rot)


class DeepSeekMoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared_experts = nn.ModuleList([Expert() for _ in range(config.n_shared)])
        self.routed_experts = nn.ModuleList([Expert() for _ in range(config.n_experts)])
        self.gate = nn.Linear(config.d_model, config.n_experts)
        self.aux_loss = 0.0

    def forward(self, x):
        # Shared experts process all tokens
        shared_out = sum(expert(x) for expert in self.shared_experts)

        # Device-limited routing
        routed_logits = self.gate(x)
        probs = F.softmax(routed_logits, dim=-1)
        topk_probs, topk_indices = probs.topk(config.top_k, dim=-1)

        # Expert balance loss
        expert_counts = torch.zeros(config.n_experts, device=x.device)
        expert_counts.scatter_add_(0, topk_indices.view(-1),
                                 torch.ones_like(topk_indices.view(-1), dtype=torch.float))
        self.aux_loss += expert_counts.float().var() * 0.003  # α1 from paper

        # Sparse computation
        routed_out = torch.zeros_like(x)
        for k in range(config.top_k):
            expert_mask = topk_indices[..., k]
            expert_contrib = torch.zeros_like(x)

            for expert_idx in range(config.n_experts):
                mask = (expert_mask == expert_idx)
                if mask.any():
                    expert_out = self.routed_experts[expert_idx](x[mask])
                    expert_contrib[mask] = expert_out * topk_probs[..., k][mask].unsqueeze(-1)

            routed_out += expert_contrib

        return shared_out + routed_out

class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.d_model)
        self.attn = MemoryOptimizedMLA()
        self.norm2 = nn.LayerNorm(config.d_model)
        self.moe = DeepSeekMoE()

    def forward(self, x, past_kv=None):
        # Attention with KV cache
        attn_out, new_kv = checkpoint(self.attn, self.norm1(x), past_kv)
        x = x + attn_out

        # MoE with checkpointing
        moe_out = checkpoint(self.moe, self.norm2(x))
        x = x + moe_out

        return x, new_kv

class DeepSeekV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(config.vocab_size, config.d_model)
        self.blocks = nn.ModuleList([TransformerBlock() for _ in range(config.n_layers)])
        self.norm = nn.LayerNorm(config.d_model)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size)

        # Better initialization with residual scaling
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_normal_(p, gain=0.1/math.sqrt(config.n_layers))
        # Add residual scaling
        for block in self.blocks:
            block.attn.output.weight.data.mul_(0.1)
            block.moe.shared_experts[0].w2.weight.data.mul_(0.1)

    def forward(self, input_ids):
        x = self.embed(input_ids)
        total_aux_loss = 0.0

        for block in self.blocks:
            x, _ = block(x)
            total_aux_loss += block.moe.aux_loss

        return self.lm_head(self.norm(x)), total_aux_loss

def train(epochs):
    model = DeepSeekV2().cuda()
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.1)

    # Learning rate schedule with warmup
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=3e-4,  # Increased max LR
        total_steps=epochs,
        pct_start=0.1,  # Shorter warmup
    )

    for epoch in range(epochs):
        # Use structured inputs
        inputs = torch.randint(0, config.vocab_size // 10,
                               (config.batch_size, config.seq_len + 1)).cuda()

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            logits, aux_loss = model(inputs[:, :-1])
            loss = F.cross_entropy(logits.view(-1, config.vocab_size),
                                   inputs[:, 1:].contiguous().view(-1))
            loss += 0.0001 * aux_loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        lr_scheduler.step()

        print(f"Epoch {epoch} Loss: {loss.item():.4f}")


if __name__ == "__main__":
    train(40)

<ipython-input-1-0d5fc762c9a9>:225: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch 0 Loss: 10.3743
Epoch 1 Loss: 10.3709
Epoch 2 Loss: 10.3212
Epoch 3 Loss: 10.2011
Epoch 4 Loss: 10.0721
Epoch 5 Loss: 10.0883
Epoch 6 Loss: 9.8448
Epoch 7 Loss: 9.8917
Epoch 8 Loss: 9.9292
Epoch 9 Loss: 10.1116
Epoch 10 Loss: 9.7340
Epoch 11 Loss: 9.6669
Epoch 12 Loss: 9.7451
Epoch 13 Loss: 9.9550
Epoch 14 Loss: 9.4845
Epoch 15 Loss: 9.7672
Epoch 16 Loss: 9.7179
Epoch 17 Loss: 9.5093
Epoch 18 Loss: 9.4009
Epoch 19 Loss: 9.4106
Epoch 20 Loss: 9.3038
Epoch 21 Loss: 9.3128
Epoch 22 Loss: 9.4516
Epoch 23 Loss: 9.2486
Epoch 24 Loss: 9.1545
Epoch 25 Loss: 9.3491
Epoch 26 Loss: 9.3559
Epoch 27 Loss: 9.2436
Epoch 28 Loss: 9.2613
Epoch 29 Loss: 9.1063
Epoch 30 Loss: 9.2779
Epoch 31 Loss: 9.2745
Epoch 32 Loss: 9.0411
Epoch 33 Loss: 9.1837
Epoch 34 Loss: 9.1079
Epoch 35 Loss: 9.0109
Epoch 36 Loss: 9.2038
Epoch 37 Loss: 9.2967
Epoch 38 Loss: 9.0990
Epoch 39 Loss: 9.0388
